In [1]:
import trainer
import torch
import random
import numpy as np
import os

from dataset import SeismogramDataset
from neural_networks.segnet import SegNet_3Head

In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
ROOT_DIR = os.path.join(os.getcwd())

In [4]:
dataset_path = os.path.join(ROOT_DIR, 'dolfin_adjoint', '2_subdomains')
train_dataset = SeismogramDataset(dataset_path)

In [5]:
from utils import number_of_parameters

model = SegNet_3Head()
number_of_parameters(model)

Total number of parameters: 4567619
Trainable number of parameters: 4567619


In [6]:
model

SegNet_3Head(
  (encoder): Sequential(
    (0): Sequential(
      (0): AdaptiveAvgPool2d(output_size=(128, 128))
      (1): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
      (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): CELU(alpha=1.0)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): CELU(alpha=1.0)
      (3): Dropout2d(p=0.0, inplace=False)
      (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
      (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (6): CELU(alpha=1.0)
      (7): Dropout2d(p=0.0, inplace=False)
      (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
      (9): InstanceNorm

In [7]:
from trainer import BaseTrainer

# NB: such format for coordinates is required for dolfin_adjoint to 
# pass grads through indexing. Why? God knows

detector_coords = [(np.array([c, 2000.])) for c in np.linspace(0., 2000., 128)]

# TODO: test with nonempty logger
t = BaseTrainer(model,
                device,
                train_dataset,
                optimizer_type=torch.optim.Adam,
                optimizer_params={'lr': 1e-3},
                snapshot_interval=10)

Found /home/dev/lambda/rs/pipeline/snapshots as a non-empty directory with 7 files


In [8]:
#TODO: test if everything is allright with batch_size > 1
t.train(detector_coords, batch_size=1, epochs=100, num_solver_type='dolfin_adjoint')

Running training procedure:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

epoch: 0; loss: 0.4835641420223258


Epoch 2 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

epoch: 1; loss: 0.4574935107067172


...

epoch: 8; loss: 0.2721364375127911


Epoch 10 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

epoch: 9; loss: 0.2612654591183885


Epoch 11 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 